## The Clever Programmer

The Clever Programmer is a blog dedicated to teaching machine lerning topics  

Is was discovered in a Medium.com article  
- https://medium.com/the-innovation/130-machine-learning-projects-solved-and-explained-605d188fb392
- https://thecleverprogrammer.com/

In [1]:
# imports
import requests
import json
import os
import sys
import time
import pandas as pd
import numpy as np
import platform
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

In [2]:
# let's speed up the scraping proccess by scraping multiple urls in one batch

# generic url scraper

def url_scraper_batch(
                urlbase = '',
                urls = [],
                selenium = True,
                windowSize = "1280,720",
                headless = True,
                quitOnEnd = True,
                waitForElement = '',
                waitForId = '',
                waitForClass = '',
                waitForIframeById = '',
                pressLink = [],
                waitBetweenPress = 30,
                waitUntilTimeout = 20,
               ):
    #meta = {
    #    'url': url
    #}
    
    # simple version
    if selenium == False:
        #page = requests.get(url)
        #meta['status'] = page.status_code
        #html = page.content
        print('batched version is for selenium only')
        return ''
      
    # selenium version
    if selenium == True:
        CHROMEDRIVER_PATH = './.selenium/chromedriver'
        WINDOW_SIZE = windowSize

        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

        driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
        driver.implicitly_wait(waitBetweenPress)
        
        result = []
        for url in urls:
            try:
                item = {
                    'url': url
                }

                driver.get(urlbase + url)

                # wait for a given tag to be loaded (javascript generated code)
                if waitForElement != '':
                    try:
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.TAG_NAME, waitForElement)))
                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("element - loading took too much time!", urlbase + url)

                # wait for a given ID to be loaded (javascript generated code)
                if waitForId != '':
                    try:
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.ID, waitForId)))
                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("id - loading took too much time!", urlbase + url)

                # wait for a given class to be loaded (javascript generated code)
                if waitForClass != '':
                    try:
                        # not-found
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.CLASS_NAME, waitForClass)))
                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("class - loading took too much time!", urlbase + url)

                # wait for a given ID to be loaded (javascript generated code)
                if waitForIframeById != '':
                    try:
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.presence_of_element_located((By.ID, waitForIframeById)))

                        driver.switch_to.frame(element)

                        item['iframe'] = driver.page_source

                        driver.switch_to.default_content()

                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("iframe - loading took too much time!", urlbase + url)

                # press link
                if len(pressLink)>0:
                    driver.implicitly_wait(waitBetweenPress)
                    try:
                        for link in pressLink:
                            print(link)
                            element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.LINK_TEXT, link)))
                            element.click()
                    except TimeoutException:
                        print ("link - loading took too much time!", urlbase + url)

                item['html'] = driver.page_source
                result.append(item)
                
            except Exception as e:
                print("Oops!", e.__class__, "occurred.")
                print(e)
                #break
            
            
        if quitOnEnd:
            driver.close()
    
    return result

urlbase = ''
urls = [
    'https://thecleverprogrammer.com/2020/11/15/machine-learning-projects/',
]

result = url_scraper_batch('', urls, waitForClass='ct-image-container', headless = False, quitOnEnd = True)
#print(result)
for item in result:
    print(item['url'], len(item['html']))
    #print(len(item['iframe']))

Page is ready! https://thecleverprogrammer.com/2020/11/15/machine-learning-projects/
https://thecleverprogrammer.com/2020/11/15/machine-learning-projects/ 161438


In [3]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

#store_data(result[0]['html'], '../data/repositories/kaggle/notebook.html')
#store_data(result[0]['iframe'], '../data/repositories/kaggle/kernel.html')

In [4]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [17]:
# get all sites from csv and store them for later feature scraping

# datasets
csv = '../data/repositories/thecleverprogrammer/tcp_index.csv'
folder = '../data/repositories/thecleverprogrammer/index/'

df = pd.read_csv(csv, sep=';')
print(df.shape)
df_links = list(df['Link'])
#chunk = df_links
batch_size = 25

chunks = [df_links[i:i + batch_size] for i in range(0, len(df_links), batch_size)]
chunks_len = len(chunks)
print('total/chunks', len(df_links), len(chunks))

for i, chunk in enumerate(chunks):
    print('### chunk', i, '/', chunks_len, '###')
    
    # check if chunk is already scraped
    chunk_dataset = []
    for j, link in enumerate(chunk):
        print('# chunk', i, '/', chunks_len, '# item ', j, link)

        temp = link.split('/')
        name = temp[-2] + '.html'
        #name = name.replace(' ','_').replace(':','')
        #print(name)
        #sys.exit()
    
        # check datasets
        file = os.path.join(folder, name)
        if not os.path.isfile(file):
            chunk_dataset.append(link)
    
    # get content for chunk
    if len(chunk_dataset) > 0:
        result = url_scraper_batch(urlbase, chunk_dataset, waitForClass='ct-image-container', headless = True, quitOnEnd = True)
    
        for item in result:
            print(item['url'], len(item['html']))
            temp = item['url'].split('/')
            name = temp[-2] + '.html'
            file = folder + name
            print(file)
            store_data(item['html'], file)



(18, 2)
total/chunks 18 1
### chunk 0 / 1 ###
# chunk 0 / 1 # item  0 https://thecleverprogrammer.com/2020/11/15/machine-learning-projects/
# chunk 0 / 1 # item  1 https://thecleverprogrammer.com/2020/11/24/all-topics-of-machine-learning-with-python/
# chunk 0 / 1 # item  2 https://thecleverprogrammer.com/2020/12/09/machine-learning-projects-on-computer-vision/
# chunk 0 / 1 # item  3 https://thecleverprogrammer.com/2020/12/07/machine-learning-projects-on-nlp/
# chunk 0 / 1 # item  4 https://thecleverprogrammer.com/2020/12/06/machine-learning-projects-on-time-series-forecasting/
# chunk 0 / 1 # item  5 https://thecleverprogrammer.com/2020/12/05/python-projects/
# chunk 0 / 1 # item  6 https://thecleverprogrammer.com/2020/12/04/machine-learning-projects-for-portfolio/
# chunk 0 / 1 # item  7 https://thecleverprogrammer.com/2020/12/03/object-detection-projects-with-python/
# chunk 0 / 1 # item  8 https://thecleverprogrammer.com/2020/12/02/web-scraping-projects-with-python/
# chunk 0 / 1 

In [29]:
# scrape links from notebooks.html

folder = '../data/repositories/thecleverprogrammer/index/'
file = 'machine-learning-projects.html'
out = 'projects.json'
#url = 'https://thecleverprogrammer.com/2020/11/15/machine-learning-projects/'

def scrape_project_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []
    
    partial = soup.find('div', class_="entry-content")
    if partial == None:
        return []
    items = partial.find_all('a') #.find_all('a', {"role": "listitem"})
    #print(len(items))
    
    for item in items:
        link = item.get('href')
        result.append(link)
        #print(link)
    
    return result

links = []

# run once to test
#html = load_data(folder+file)
#links = scrape_project_links(html)

files = os.listdir(folder)
for file in files:
    html = load_data(folder+file)
    result = scrape_project_links(html)
    links.extend(result)

print('len raw:', len(links))

# make links unique
links = list(set(links))
print('len unique:', len(links))

# clean links / only keep links to https://thecleverprogrammer.com/
for link in links:
    if not 'thecleverprogrammer.com' in link:
        links.remove(link)
print('len internal:', len(links))

store_data(links, folder+out, toJson=True)

len raw: 536
len unique: 322
len internal: 316


26669

In [31]:
# download projects

file = '../data/repositories/thecleverprogrammer/index/projects.json'
path = '../data/repositories/thecleverprogrammer/projects/html/'
urlbase = ''

df_links = load_data(file, fromJson=True)
print(len(df_links))
batch_size = 25

chunks = [df_links[i:i + batch_size] for i in range(0, len(df_links), batch_size)]
chunks_len = len(chunks)
print('total/chunks', len(df_links), len(chunks))

for i, chunk in enumerate(chunks):
    print('### chunk', i, '/', chunks_len, '###')
    
    # check if chunk is already scraped
    chunk_dataset = []
    for j, link in enumerate(chunk):
        print('# chunk', i, '/', chunks_len, '# item ', j, link)

        name = link.replace('https://thecleverprogrammer.com/','').replace('/','_') + '.html'
        name = name.replace('_.html','.html')
        print(name)
        #sys.exit()
    
        # check datasets
        file = os.path.join(folder, name)
        if not os.path.isfile(file):
            chunk_dataset.append(link)
    
    # get content for chunk
    if len(chunk_dataset) > 0:
        result = url_scraper_batch(urlbase, chunk_dataset, waitForClass='ct-image-container', headless = True, quitOnEnd = True)
    
        for item in result:
            print(item['url'], len(item['html']))
            name = item['url'].replace('https://thecleverprogrammer.com/','').replace('/','_') + '.html'
            name = name.replace('_.html','.html')
            file = path + name
            print(file)
            store_data(item['html'], file)



316
total/chunks 316 13
### chunk 0 / 13 ###
# chunk 0 / 13 # item  0 https://thecleverprogrammer.com/2020/11/05/cnn-architectures-in-machine-learning/
2020_11_05_cnn-architectures-in-machine-learning.html
# chunk 0 / 13 # item  1 https://thecleverprogrammer.com/2020/09/23/analyze-ipl-with-python/
2020_09_23_analyze-ipl-with-python.html
# chunk 0 / 13 # item  2 https://thecleverprogrammer.com/2020/11/07/machine-learning-in-5-lines-with-mindsdb/
2020_11_07_machine-learning-in-5-lines-with-mindsdb.html
# chunk 0 / 13 # item  3 https://thecleverprogrammer.com/2020/11/25/sentiment-analysis-projects-with-python/
2020_11_25_sentiment-analysis-projects-with-python.html
# chunk 0 / 13 # item  4 https://thecleverprogrammer.com/2020/11/22/convert-categorical-features-to-numerical-with-python/
2020_11_22_convert-categorical-features-to-numerical-with-python.html
# chunk 0 / 13 # item  5 https://thecleverprogrammer.com/2020/10/22/create-an-audiobook-with-python/
2020_10_22_create-an-audiobook-with

Page is ready! https://thecleverprogrammer.com/2020/05/26/data-science-project-gdp-analysis/
Page is ready! https://thecleverprogrammer.com/2020/11/09/plastic-users-analysis-with-python/
Page is ready! https://thecleverprogrammer.com/2020/10/15/what-are-decision-trees-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/05/17/data-science-project-email-spam-detection-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/29/fake-currency-detection-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/20/maths-for-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/08/what-is-big-data/
Page is ready! https://thecleverprogrammer.com/2020/09/22/predict-ipl-winner-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/07/19/wordcloud-with-python/
Page is ready! https://thecleverprogrammer.com/2020/11/30/amazon-bestselling-books-analysis-with-python/
Page is ready! https://thecleverpr

Page is ready! https://thecleverprogrammer.com/2020/05/27/human-activity-recognition-using-smartphone-data-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/11/what-is-a-recommendation-system-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/14/will-automl-replace-data-science-jobs/
Page is ready! https://thecleverprogrammer.com/2020/07/19/image-classification-with-ann/
Page is ready! https://thecleverprogrammer.com/2020/09/07/scaling-and-normalization-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/11/word-embeddings-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/31/text-classification-with-tensorflow-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/07/12/employee-turnover-prediction/
Page is ready! https://thecleverprogrammer.com/2020/08/29/machine-learning-interview-questions/
Page is ready! https://thecleverprogrammer.com/2020/10/12/statistics-for-m

Page is ready! https://thecleverprogrammer.com/2020/09/04/xgboost-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/09/ridge-and-lasso-regression-with-python/
Page is ready! https://thecleverprogrammer.com/2020/11/18/restaurant-recommendation-system-with-python/
Page is ready! https://thecleverprogrammer.com/2020/08/04/fraud-detection-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/31/dbscan-algorithm-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/05/09/data-science-project-on-handwritten-digits/
Page is ready! https://thecleverprogrammer.com/2020/05/09/data-science-project-on-text-and-annotations/
Page is ready! https://thecleverprogrammer.com/2020/08/25/python-projects-for-beginners/
Page is ready! https://thecleverprogrammer.com/2020/09/05/what-are-neural-networks-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/03/random-forest-algorithm/
Page is ready! https://thecleverprog

Page is ready! https://thecleverprogrammer.com/2020/12/01/keyword-extraction-with-python/
Page is ready! https://thecleverprogrammer.com/2020/08/27/daily-births-forecasting-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/26/image-recognition-with-machine-learning-using-pytorch/
Page is ready! https://thecleverprogrammer.com/2020/07/09/covid-19-detection/
Page is ready! https://thecleverprogrammer.com/2020/11/01/chatbot-with-machine-learning-and-python/
Page is ready! https://thecleverprogrammer.com/2020/08/12/network-security-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/05/08/data-science-project-on-birth-rate-analysis/
Page is ready! https://thecleverprogrammer.com/2020/07/20/data-augmentation-in-deep-learning/
Page is ready! https://thecleverprogrammer.com/2020/05/22/data-science-project-fake-news-classification/
Page is ready! https://thecleverprogrammer.com/2020/08/08/oop-for-machine-learning/
Page is ready! https://thecle

...

Page is ready! https://thecleverprogrammer.com/2020/11/01/what-is-hadoop-in-data-science/
Page is ready! https://thecleverprogrammer.com/2020/09/18/choose-algorithm-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/31/python-libraries-for-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/21/knn-algorithm-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/06/adaboost-algorithm/
Page is ready! https://thecleverprogrammer.com/2020/09/07/neural-network-with-python-code/
Page is ready! https://thecleverprogrammer.com/2020/11/17/face-mask-detection-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/05/covid-19-analysis-with-python/
Page is ready! https://thecleverprogrammer.com/2020/10/25/what-is-cross-validation-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/09/face-detection-with-python/
Page is ready! https://thecleverprogrammer.com/2020/07/09/training-and-test

Page is ready! https://thecleverprogrammer.com/2020/06/13/twitter-sentiment-analysis/
Page is ready! https://thecleverprogrammer.com/2020/09/05/data-leakage-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/24/data-cleaning-with-python/
Page is ready! https://thecleverprogrammer.com/2020/05/23/data-science-project-book-recommendation-system-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/11/02/types-of-machine-learning-algorithms/
Page is ready! https://thecleverprogrammer.com/2020/09/21/predict-car-prices-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/05/23/book-recommendation-system-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/01/image-segmentation-with-python/
Page is ready! https://thecleverprogrammer.com/2020/05/11/data-science-project-stock-price-prediction-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/16/what-is-the-support-vector-mach

Page is ready! https://thecleverprogrammer.com/2020/08/19/hate-speech-detection-model/
Page is ready! https://thecleverprogrammer.com/2020/05/08/data-science-project-on-time-series/
Page is ready! https://thecleverprogrammer.com/2020/07/10/reinforcement-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/09/nlp-for-other-languages-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/05/26/data-science-project-supermarket-sales-analysis/
Page is ready! https://thecleverprogrammer.com/2020/04/23/python-gui-app-for-calendar/
Page is ready! https://thecleverprogrammer.com/2020/07/06/support-vector-machines-svm-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/11/24/flower-recognition-with-python/
Page is ready! https://thecleverprogrammer.com/2020/08/02/gender-classification-model/
Page is ready! https://thecleverprogrammer.com/2020/08/28/binary-search-algorithm-with-python/
Page is ready! https://thecleverprogrammer.com/2020/10/24/

Page is ready! https://thecleverprogrammer.com/2020/11/20/hand-gesture-recognition-with-python/
Page is ready! https://thecleverprogrammer.com/2020/11/29/covid-19-cases-prediction-with-python/
Page is ready! https://thecleverprogrammer.com/2020/08/09/stock-price-prediction-with-facebook-prophet-model/
Page is ready! https://thecleverprogrammer.com/2020/11/20/linear-regression-with-python/
Page is ready! https://thecleverprogrammer.com/2020/05/26/data-science-project-san-francisco-crime-analysis/
Page is ready! https://thecleverprogrammer.com/2020/05/23/bitcoin-price-prediction-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/12/ab-testing-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/11/launch-a-machine-learning-model/
Page is ready! https://thecleverprogrammer.com/2020/09/19/analyze-call-records-with-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/09/03/abc-analysis-with-machine-learning/
Page is ready! h

Page is ready! https://thecleverprogrammer.com/2020/08/15/visualize-geospatial-data/
Page is ready! https://thecleverprogrammer.com/2020/12/04/number-guessing-game-with-python-and-c/
Page is ready! https://thecleverprogrammer.com/2020/10/04/card-game-with-python/
Page is ready! https://thecleverprogrammer.com/2020/09/17/nlp-projects-for-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/07/08/manifold-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/11/machine-learning-in-finance/
Page is ready! https://thecleverprogrammer.com/2020/04/30/python-gui-app-for-student-details/
Page is ready! https://thecleverprogrammer.com/2020/11/06/roadmap-for-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/10/09/graph-algorithms-with-python/
Page is ready! https://thecleverprogrammer.com/2020/08/04/arima-model-in-machine-learning/
Page is ready! https://thecleverprogrammer.com/2020/08/30/image-classification-with-tensorflow-in-machine-learning/
Pa

In [5]:
# scan text for predefined terms

text = 'We use LSTM for anomaly and object detection. As Convolutional Neural Networks are great for ML.'

pd_ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
ml_terms = pd_ml_terms['Term'].tolist()
ml_slugs = pd_ml_terms['Slug'].tolist()
ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
ml_tags = pd_ml_terms['Tag'].tolist()
ml_tags = [x for x in ml_tags if str(x) != 'nan']

#print(ml_tags)

ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
ml_libs = ml_libs['Python Package'].tolist()

def match_text(haystack, needles, toLower = False, unique = True):
    
    if toLower == True:
        haystack = haystack.lower()
        needles = [x.lower() for x in needles]
    
    if unique == True:
        matches = {x for x in needles if x in haystack}
        matches = list(matches)
    else:
        matches = [x for x in needles if x in haystack]
    
    return matches

def match_tags(haystack):
    df = pd.read_csv('../data/patterns/ml_terms.csv')
    tags = []
    
    df.set_index('Term', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(tag)
        except:
            pass
        
    df.set_index('Slug', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(str(tag))
        except:
            pass
        
    #if 'ANN' in tags or 'CNN' in tags or 'RNN' in tags:
    #    tags.remove('NN')
    
    return list(set(tags))

#ml_slugs, ml_terms, ml_libs, match_text(haystack, needles, toLower = False, unique = True)
needles = {
    'ml_slugs': ml_slugs,
    'ml_terms': ml_terms,
    'ml_libs': ml_libs,
}
needles_need_str_lower = {
    'ml_slugs': False,
    'ml_terms': True,
    'ml_libs': False,
}

matches = []

matches.extend(match_text(text, ml_terms, True))
matches.extend(match_text(text, ml_slugs, False))
print('matches', matches)

tags = match_tags(matches)
print('tags', tags)

matches ['detect', 'neural network', 'anomaly', 'object detection', 'convolutional neural network', 'lstm', 'ML']
tags ['LSTM', 'NN', 'Object Detection', 'ML', 'CNN']


In [6]:
# get file modifictaion date
# https://stackoverflow.com/questions/237079/how-to-get-file-creation-modification-date-times-in-python

def creation_date(path_to_file, datetime = True):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        timestamp = os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            timestamp = stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            timestamp = stat.st_mtime
        
    if datetime == True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

    return timestamp
    
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'
print(creation_date(folder_base+folder+notebook))

2020-12-12 16:08:03


In [42]:
# clear text formatting
import re
def clear_text(s):
    s = s.replace('\n',' ').replace('\r','').replace('¶','').strip()
    s = re.sub("\s\s+" , " ", s)
    return s

print(clear_text("The   fox jumped   over    the log."))

The fox jumped over the log.


In [44]:
# scrape project content

path_in  = '../data/repositories/thecleverprogrammer/projects/html/'
path_out = '../data/repositories/thecleverprogrammer/projects/meta/'

test_file = '2020_08_04_named-entity-recognition-ner.html'

base_url = 'https://thecleverprogrammer.com/'

def scrape_project_content(html, url, needles):
    
    
    soup = BeautifulSoup(html, 'html.parser')
    
    snippet = soup.find('div', class_='entry-content')
    
    # remove 'also read' block
    classes = ['wp-block-coblocks-highlight']
    for i in classes:
        for elem in snippet.find_all("p", {'class': i}):
            elem.decompose()
    
    #code = snippet.find_all('code')
    #code = [x.text.strip() for x in code]
    #code = '\n'.join(code)
    
    #try:
    #    text = snippet.pre.decompose()
    #    text = snippet.div.decompose()
    #except:
    #    print('can not reduce text')
    text = snippet.text
    
    meta = {}
    
    meta['title'] = soup.find('h1', class_="page-title").text
    meta['link'] = url
    meta['date'] = soup.find('time', class_="ct-meta-element-date").get('datetime')

    # date formatting #"2020-05-08T19:41:42+05:30"
    # ignore "+05:30"
    date_time_str = meta['date'].split('+')
    date_time_str = date_time_str[0]
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S')
    meta['date'] = str(date_time_obj)
        
    meta['text'] = clear_text(text)
    #meta['code'] = code
    
    #meta = {}
    
    for key, value in needles.items():
        #if 'ml_libs' in key:
        #    meta[key] = match_text(code, value, needles_need_str_lower[key])
        #else:
        meta[key] = match_text(text, value, needles_need_str_lower[key])
        
    meta['description'] = ''
    for item in snippet.find_all(['p','h2']):
        if item.name == 'p':
            meta['description'] += item.text
        elif item.name == 'h2':
            break
        
    meta['ml_tags'] = match_tags(meta['ml_terms'] + meta['ml_slugs'])
    
    # ml score
    ml_score = 0
    if len(meta['ml_slugs']) > 0 or len(meta['ml_terms']) > 0:
        ml_score += 0.2
    if len(meta['ml_tags']) > 0:
        ml_score += 0.3
    if len(meta['ml_libs']) > 0:
        ml_score += 0.5
    meta['ml_score'] = ml_score
    
    return meta

# run once or for all files
test = False

if test:
    html = load_data(path_in+test_file)
    if 'file not found' in html:
        print(html)
    url = base_url + test_file.replace('_','/').replace('.html','/')
    meta = scrape_project_content(html, url, needles)
    print(meta)
    
else:
    files = os.listdir(path_in)
    for i, file in enumerate(files):
        print('#', i, file)
        html = load_data(path_in+file)
        if 'file not found' in html:
            print(html)
            break
        url = base_url + file.replace('_','/').replace('.html','/')
        meta = scrape_project_content(html, url, needles)
        
        file_out = file.replace('html','json')
        store_data(meta, path_out+file_out, toJson=True)

# 0 2020_04_23_python-gui-app-for-calendar.html
# 1 2020_04_30_python-gui-app-for-student-details.html
# 2 2020_05_05_creating-graphics-with-python.html
# 3 2020_05_08_data-science-project-on-birth-rate-analysis.html
# 4 2020_05_08_data-science-project-on-president-heights.html
# 5 2020_05_08_data-science-project-on-time-series.html
# 6 2020_05_09_data-science-project-on-area-and-population.html
# 7 2020_05_09_data-science-project-on-handwritten-digits.html
# 8 2020_05_09_data-science-project-on-text-and-annotations.html
# 9 2020_05_09_python-kivy-tutorials-introduction-to-kivy.html
# 10 2020_05_10_python-clock-app-with-kivy.html
# 11 2020_05_11_data-science-project-stock-price-prediction-with-machine-learning.html
# 12 2020_05_11_stock-price-prediction-with-machine-learning.html
# 13 2020_05_14_data-science-project-on-classification-of-text.html
# 14 2020_05_15_amazon-product-reviews-sentiment-analysis-with-machine-learning.html
# 15 2020_05_15_data-science-project-on-extracting-hog-f

# 140 2020_08_24_summarize-text-with-machine-learning.html
# 141 2020_08_25_bert-in-machine-learning.html
# 142 2020_08_25_python-projects-for-beginners.html
# 143 2020_08_25_titanic-survival-with-machine-learning.html
# 144 2020_08_26_image-recognition-with-machine-learning-using-pytorch.html
# 145 2020_08_26_linear-algebra-for-machine-learning.html
# 146 2020_08_27_daily-births-forecasting-with-machine-learning.html
# 147 2020_08_27_data-science-resume.html
# 148 2020_08_27_difference-between-algorithm-and-model-in-machine-learning.html
# 149 2020_08_27_one-hot-encoding-in-machine-learning.html
# 150 2020_08_28_binary-search-algorithm-with-python.html
# 151 2020_08_28_galaxy-classification-with-machine-learning.html
# 152 2020_08_29_machine-learning-interview-questions.html
# 153 2020_08_29_structured-data-and-unstructured-data-in-machine-learning.html
# 154 2020_08_29_time-series-with-lstm-in-machine-learning.html
# 155 2020_08_30_how-to-create-a-package-with-python.html
# 156 2020_

# 280 2020_11_18_what-is-feature-engineering-in-machine-learning.html
# 281 2020_11_20_hand-gesture-recognition-with-python.html
# 282 2020_11_20_how-to-reduce-overfitting-in-machine-learning.html
# 283 2020_11_20_linear-regression-with-python.html
# 284 2020_11_20_target-variable-in-machine-learning.html
# 285 2020_11_21_covid-19-projects-with-python-and-machine-learning.html
# 286 2020_11_21_employee-attrition-prediction-with-python.html
# 287 2020_11_21_how-much-training-data-is-required-for-machine-learning.html
# 288 2020_11_22_convert-categorical-features-to-numerical-with-python.html
# 289 2020_11_22_deep-learning-projects-with-python.html
# 290 2020_11_22_pneumonia-detection-with-python.html
# 291 2020_11_23_air-quality-index-analysis-with-python.html
# 292 2020_11_23_machine-learning-process.html
# 293 2020_11_23_recommendation-system-projects-with-python.html
# 294 2020_11_24_flower-recognition-with-python.html
# 295 2020_11_25_gender-classification-with-python.html
# 296 202

In [46]:
# throw all parsed meta-data together in a single csv
# select only true ML cases

path_in = '../data/repositories/thecleverprogrammer/projects/meta/'
csv_out = '../data/database/thecleverprogrammer_01_original.csv'

quit = 0 # quit after n files processed / 0 ... no limit

files = os.listdir(path_in)
print('files:', len(files))
i = 0

runtime_start = time.time()
df = pd.DataFrame()

for file in files:
    #print('#', i, file)
    path = os.path.join(path_in, file)
    i += 1

    #print(' - ', j, 'author:', author, '/ notebook:', notebook)
    data = load_data(path, fromJson=True)
    #print(data)
    
    data['text'] = clear_text(data['text'])
    
    # store only items with:
    # - ml_score >= 0.5
    if data['ml_score'] >= 0.5 and len(data['ml_libs']) > 0:
        if not 'all-machine-learning-algorithms-explained' in data['link']:
            df = df.append(data, ignore_index=True)
            #print(data['link'], 'is ML use case')

    if quit!=0 and j>quit:
        break
        
# drop duplicates
df = df.drop_duplicates(['link'])

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', i, 'items')
print(df.shape)
print(df.head())
        
# drop columns
#df.drop(columns=['code', 'text'], inplace=True)
#df.drop(columns=['text'], inplace=True)

df.to_csv(csv_out, sep=';', index=False)
print('done')

files: 316
runtime: 0.88 seconds for 316 items
(172, 10)
                  date                                        description  \
0  2020-05-09 19:40:35  Let’s consider one piece of the optical charac...   
1  2020-05-11 13:57:50  In this Data Science Project we will create a ...   
2  2020-05-11 13:57:50  In this Data Science Project we will create a ...   
3  2020-05-14 17:30:05  One place in Data Science where multinomial na...   
4  2020-05-15 13:10:55  Product reviews are becoming more important wi...   

                                                link    ml_libs  ml_score  \
0  https://thecleverprogrammer.com/2020/05/09/dat...  [sklearn]       1.0   
1  https://thecleverprogrammer.com/2020/05/11/dat...  [sklearn]       1.0   
2  https://thecleverprogrammer.com/2020/05/11/sto...  [sklearn]       1.0   
3  https://thecleverprogrammer.com/2020/05/14/dat...  [sklearn]       1.0   
4  https://thecleverprogrammer.com/2020/05/15/ama...  [sklearn]       1.0   

  ml_slugs       